In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch

In [2]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

from MlpTSMixer import MlpTSMixerEstimator

/home/kashif/.env/pytorch/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/kashif/.env/pytorch/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
dataset = get_dataset("solar_nips", regenerate=False)
train_grouper = MultivariateGrouper(
    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality)
)

test_grouper = MultivariateGrouper(
    num_test_dates=int(len(dataset.test) / len(dataset.train)),
    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
)
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

In [7]:
estimator = MlpTSMixerEstimator(
    # distr_output=StudentTOutput(dim=int(dataset.metadata.feat_static_cat[0].cardinality)),
    input_size=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    context_length=dataset.metadata.prediction_length * 3,
    freq=dataset.metadata.freq,
    scaling="std",
    pooling_type="max",
    depth=2,
    patch_size=(5, 5),
    dim=32,
    expansion_factor=1,
    batch_size=64,
    num_batches_per_epoch=100,
    trainer_kwargs=dict(accelerator="cuda", max_epochs=100),
)

In [8]:
predictor = estimator.train(
    training_data=dataset_train,
    cache_data=True,
    shuffle_buffer_length=1024,
)

Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/kashif/.env/pytorch/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Ty

Training: 0it [00:00, ?it/s]

Epoch 0, global step 100: 'train_loss' reached 4.29688 (best 4.29688), saving model to '/mnt/scratch/kashif/ConvTS-Mixer/lightning_logs/version_142/checkpoints/epoch=0-step=100.ckpt' as top 1
Epoch 1, global step 200: 'train_loss' reached 3.79278 (best 3.79278), saving model to '/mnt/scratch/kashif/ConvTS-Mixer/lightning_logs/version_142/checkpoints/epoch=1-step=200.ckpt' as top 1
Epoch 2, global step 300: 'train_loss' reached 3.68673 (best 3.68673), saving model to '/mnt/scratch/kashif/ConvTS-Mixer/lightning_logs/version_142/checkpoints/epoch=2-step=300.ckpt' as top 1
Epoch 3, global step 400: 'train_loss' reached 3.62648 (best 3.62648), saving model to '/mnt/scratch/kashif/ConvTS-Mixer/lightning_logs/version_142/checkpoints/epoch=3-step=400.ckpt' as top 1
Epoch 4, global step 500: 'train_loss' was not in top 1
Epoch 5, global step 600: 'train_loss' reached 3.60439 (best 3.60439), saving model to '/mnt/scratch/kashif/ConvTS-Mixer/lightning_logs/version_142/checkpoints/epoch=5-step=600

Epoch 69, global step 7000: 'train_loss' was not in top 1
Epoch 70, global step 7100: 'train_loss' reached 3.30263 (best 3.30263), saving model to '/mnt/scratch/kashif/ConvTS-Mixer/lightning_logs/version_142/checkpoints/epoch=70-step=7100.ckpt' as top 1
Epoch 71, global step 7200: 'train_loss' was not in top 1
Epoch 72, global step 7300: 'train_loss' reached 3.29853 (best 3.29853), saving model to '/mnt/scratch/kashif/ConvTS-Mixer/lightning_logs/version_142/checkpoints/epoch=72-step=7300.ckpt' as top 1
Epoch 73, global step 7400: 'train_loss' was not in top 1
Epoch 74, global step 7500: 'train_loss' was not in top 1
Epoch 75, global step 7600: 'train_loss' reached 3.29573 (best 3.29573), saving model to '/mnt/scratch/kashif/ConvTS-Mixer/lightning_logs/version_142/checkpoints/epoch=75-step=7600.ckpt' as top 1
Epoch 76, global step 7700: 'train_loss' reached 3.29283 (best 3.29283), saving model to '/mnt/scratch/kashif/ConvTS-Mixer/lightning_logs/version_142/checkpoints/epoch=76-step=7700

In [9]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset_test, predictor=predictor, num_samples=100
)
forecasts = list(forecast_it)
targets = list(ts_it)

In [10]:
evaluator = MultivariateEvaluator(
    quantiles=(np.arange(20) / 20.0)[1:], target_agg_funcs={"sum": np.sum}
)

In [11]:
agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

Running evaluation: 7it [00:00, 72.80it/s]
Running evaluation: 7it [00:00, 111.39it/s]
Running evaluation: 7it [00:00, 110.80it/s]
Running evaluation: 7it [00:00, 110.80it/s]
Running evaluation: 7it [00:00, 112.00it/s]
Running evaluation: 7it [00:00, 112.25it/s]
Running evaluation: 7it [00:00, 112.36it/s]
Running evaluation: 7it [00:00, 111.67it/s]
Running evaluation: 7it [00:00, 112.26it/s]
Running evaluation: 7it [00:00, 112.67it/s]
Running evaluation: 7it [00:00, 112.34it/s]
Running evaluation: 7it [00:00, 112.46it/s]
Running evaluation: 7it [00:00, 112.61it/s]
Running evaluation: 7it [00:00, 112.46it/s]
Running evaluation: 7it [00:00, 112.19it/s]
Running evaluation: 7it [00:00, 111.27it/s]
Running evaluation: 7it [00:00, 111.52it/s]
Running evaluation: 7it [00:00, 111.46it/s]
Running evaluation: 7it [00:00, 112.73it/s]
Running evaluation: 7it [00:00, 112.38it/s]
Running evaluation: 7it [00:00, 111.58it/s]
Running evaluation: 7it [00:00, 112.23it/s]
Running evaluation: 7it [00:00, 1

In [12]:
print("CRPS-Sum: {}".format(agg_metric["m_sum_mean_wQuantileLoss"]))
print("CRPS: {}".format(agg_metric["mean_wQuantileLoss"]))
print("QL50: {}".format(agg_metric["wQuantileLoss[0.5]"]))
print("QL90: {}".format(agg_metric["wQuantileLoss[0.9]"]))
print("MSIS: {}".format(agg_metric["MSIS"]))
print("NRMSE: {}".format(agg_metric["NRMSE"]))
print("sMAPE: {}".format(agg_metric["sMAPE"]))
print("MASE: {}".format(agg_metric["MASE"]))

# print("ND: {}".format(agg_metric["ND"]))
# print("NRMSE: {}".format(agg_metric["NRMSE"]))
# print("MSE: {}".format(agg_metric["MSE"]))

CRPS-Sum: 0.32429501431459606
CRPS: 0.3625486813719906
QL50: 0.4988281185856732
QL90: 0.20497564379619174
MSIS: 7.8547312171213095
NRMSE: 0.9782419894884901
sMAPE: 1.3554419422050215
MASE: 1.1668081421889942


In [70]:
print("CRPS: {}".format(agg_metric["mean_wQuantileLoss"]))
print("ND: {}".format(agg_metric["ND"]))
print("NRMSE: {}".format(agg_metric["NRMSE"]))
print("MSE: {}".format(agg_metric["MSE"]))

print("CRPS-SUM: {}".format(agg_metric["m_sum_mean_wQuantileLoss"]))

CRPS: 0.3475042523854184
ND: 0.49421386986725574
NRMSE: 0.9454882241836582
MSE: 847.98864290372
CRPS-SUM: 0.29327103321837134


In [3]:
def plot(
    target,
    forecast,
    prediction_length,
    prediction_intervals=(50.0, 90.0),
    color="g",
    fname=None,
):
    label_prefix = ""
    rows = 4
    cols = 4
    fig, axs = plt.subplots(rows, cols, figsize=(24, 24))
    axx = axs.ravel()
    seq_len, target_dim = target.shape

    ps = [50.0] + [
        50.0 + f * c / 2.0 for c in prediction_intervals for f in [-1.0, +1.0]
    ]

    percentiles_sorted = sorted(set(ps))

    def alpha_for_percentile(p):
        return (p / 100.0) ** 0.3

    for dim in range(0, min(rows * cols, target_dim)):
        ax = axx[dim]

        target[-2 * prediction_length :][dim].plot(ax=ax)

        ps_data = [forecast.quantile(p / 100.0)[:, dim] for p in percentiles_sorted]
        i_p50 = len(percentiles_sorted) // 2

        p50_data = ps_data[i_p50]
        p50_series = pd.Series(data=p50_data, index=forecast.index)
        p50_series.plot(color=color, ls="-", label=f"{label_prefix}median", ax=ax)

        for i in range(len(percentiles_sorted) // 2):
            ptile = percentiles_sorted[i]
            alpha = alpha_for_percentile(ptile)
            ax.fill_between(
                forecast.index,
                ps_data[i],
                ps_data[-i - 1],
                facecolor=color,
                alpha=alpha,
                interpolate=True,
            )
            # Hack to create labels for the error intervals.
            # Doesn't actually plot anything, because we only pass a single data point
            pd.Series(data=p50_data[:1], index=forecast.index[:1]).plot(
                color=color,
                alpha=alpha,
                linewidth=10,
                label=f"{label_prefix}{100 - ptile * 2}%",
                ax=ax,
            )

    legend = ["observations", "median prediction"] + [
        f"{k}% prediction interval" for k in prediction_intervals
    ][::-1]
    axx[0].legend(legend, loc="upper left")

    if fname is not None:
        plt.savefig(fname, bbox_inches="tight", pad_inches=0.05)

In [4]:
plot(
    target=targets[0],
    forecast=forecasts[0],
    prediction_length=dataset.metadata.prediction_length,
)
plt.show()

NameError: name 'targets' is not defined